# Figure 2

---

If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/Golden-Framework.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/Golden-Framework')

---

## Load packages

In [ ]:
import scipy.io as io
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

## Load functions for simulations and plotting

In [ ]:
from Golden_Ratio_functions import *

## Figure 2B: Target responses without sinusoidal gain.

In [ ]:
ratio   = (1+np.sqrt(5))/2
f_k     = ratio**np.arange(2,9+1)
network = {"f_k":f_k, "k_perturbed":4}
gain    = {"f_S":0, "g_S":0}

x,t = run_damped_coupled_harmonic_oscillators(network, gain)

f = plt.figure(figsize=(2, 3), dpi=80)
plot_model_traces(t,x,network,gain)
#f.savefig("./PDFs/Figure-2B.pdf", bbox_inches='tight')

## Figure 2C: Target responses with sinusoidal gain at frequency $\phi^7$ Hz.

In [ ]:
ratio   = (1+np.sqrt(5))/2
f_k     = ratio**np.arange(2,9+1)
network = {"f_k":f_k, "k_perturbed":4}
gain    = {"f_S":ratio**7, "g_S":50}

x,t = run_damped_coupled_harmonic_oscillators(network, gain)

f = plt.figure(figsize=(2, 3), dpi=80)
plot_model_traces(t,x,network,gain)
#f.savefig("./PDFs/Figure-2C.pdf", bbox_inches='tight')

### Aside: estimate frequency of response at each node.
Note: counting starts at node 0, and we expect responses at nodes 3, 4 (driven node 17.9 Hz), and 6.

In [ ]:
f = plt.figure(figsize=(20, 3), dpi=80)
plt.subplot(1,3,1); estimate_oscillator_frequency(x,t,f_k,3)  #Node 3, expect 11 Hz
plt.subplot(1,3,2); estimate_oscillator_frequency(x,t,f_k,4)  #Node 4, expect 17.9 Hz
plt.subplot(1,3,3); estimate_oscillator_frequency(x,t,f_k,6)  #Node 6, expect 47.0 Hz

## Figure 2D (Part 1): Compute resonant response versus sinusoidal gain frequency, and save it.
This part is slow, and iterates over a range of `f_S`. Results saved in a `.mat` file, available on GitHub.

In [ ]:
ratio   = (1+np.sqrt(5))/2;
f_k     = ratio**np.arange(2,9+1)
network = {"f_k":f_k, "k_perturbed":4}

f_S     = np.arange(1,100,0.1)
g_S     = 50
A       = np.zeros([16,f_S.size])

for i in np.arange(f_S.size):

    print(["%.1f"% f_S[i]][0],end=' ')
    
    gain    = {"f_S":f_S[i], "g_S":g_S}
    x,t = run_damped_coupled_harmonic_oscillators(network, gain)
    
    amp   = np.abs(hilbert(x));
    i0    = np.where( (t>=0) & (t<=1.5) );
    A[:,i]= np.mean(amp[:,i0[0]],1)

res = {"A":A, "f_S":f_S, "g_S":g_S, "f_k":f_k, "network":network, "gain":gain}
io.savemat('Figure-2D.mat', res)

## Figure 2D (Part 2): Load resonant response versus sinusoidal gain frequency, and plot it.

This part is fast.

In [ ]:
res = io.loadmat("Figure-2D.mat");
# Note: if you're running on Google Colab, then load the .mat file like this:
# res = io.loadmat("/content/Golden-Framework/Figure-2D.mat");
f   = plt.figure(figsize=(5, 3), dpi=80)
plot_gain_traces(res)
#f.savefig("./PDFs/Figure-2D.pdf", bbox_inches='tight')

## Figure 2E (Part 1): Compute impact of noise on evoked response, and save it.
This part is slow, and iterates over a range of `noise` for `replicates` times. Results saved in a `.mat` file, available on GitHub.

In [ ]:
# Compute average evoked response without noise.
ratio     = (1+np.sqrt(5))/2
f_k       = ratio**np.arange(2,9+1)
network   = {"f_k":f_k, "k_perturbed":4}
gain      = {"f_S":ratio**7, "g_S":50}
noise     = np.zeros(16)
perturbation = 1
x,t       = damped_coupled_harmonic_oscillators_with_noise(network, gain, noise, perturbation)
i0        = np.where( (t>=0) & (t<=1.5) )
sd        = np.std(x[:,i0[0]],1)
evoked_sd = np.mean(sd[[0,2,4,6,  10,12,14]])
print("Computed average evoked response ... \n")

#... and use it 
noise   = np.array([0,0.5,1.0,1.5,2])
replicates = 100;
A_yes_perturb = np.zeros([16,noise.size,replicates])
A_no_perturb  = np.zeros([16,noise.size,replicates])

for j in np.arange(noise.size):
    
    print(["%.1f"% noise[j]][0],end=': ')
    
    for k in np.arange(replicates):
    
        print(["%.0f"% k][0],end=' ')
        
        perturbation = 1                                    # Simulate with noise, and perturb a node.
        x,t   = damped_coupled_harmonic_oscillators_with_noise(network, gain, evoked_sd*np.ones(16)*noise[j], perturbation)
        amp   = np.abs(hilbert(x))                          # ... compute the amplitude,
        i0    = np.where( (t>=0) & (t<=1.5) )               # ... after the perturbation,
        A_yes_perturb[:,j,k]= np.mean(amp[:,i0[0]],1)       # ... and save it.

        perturbation = 0                                    # Simulate with noise, with *no* perturbation.
        x,t   = damped_coupled_harmonic_oscillators_with_noise(network, gain, evoked_sd*np.ones(16)*noise[j], perturbation)
        amp   = np.abs(hilbert(x))                          # ... compute the amplitude,
        i0    = np.where( (t>=0) & (t<=1.5) )               # ... after the perturbation,
        A_no_perturb[:,j,k] = np.mean(amp[:,i0[0]],1)       # ... and save it.
    
    print("\n")

res = {"A_yes_perturb":A_yes_perturb, "A_no_pertrub":A_no_perturb, "f_k":f_k, "network":network, "gain":gain, "sd":sd, "noise":noise}
io.savemat(str('Figure-2E.mat'), res)

## Figure 2E (Part 2): Load evoked response versus noise, and plot it.

This part is fast.

In [ ]:
res = io.loadmat("Figure-2E.mat")
# Note: if you're running on Google Colab, then load the .mat file like this:
# res = io.loadmat("/content/Golden-Framework/Figure-2E.mat");
f = plt.figure(figsize=(20, 3), dpi=80)
plt.subplot(1,6,1); plot_response_with_noise(res, 2); plt.ylim([0,0.03])
plt.subplot(1,6,2); plot_response_with_noise(res, 3); plt.ylim([0,0.03])
plt.subplot(1,6,3); plot_response_with_noise(res, 4)
plt.subplot(1,6,4); plot_response_with_noise(res, 5); plt.ylim([0,0.03])
plt.subplot(1,6,5); plot_response_with_noise(res, 6); plt.ylim([0,0.03])
plt.subplot(1,6,6); plot_response_with_noise(res, 7); plt.ylim([0,0.03]);
#f.savefig("./PDFs/Figure-2D.pdf", bbox_inches='tight')